# Where to construct distribution centers?
 - Correction
Consider a set of $m$ locations $\{1,\dots, m\}$ in which a distribution center can be open.
In addition, consider a set of $n$ clients $\{1,\dots, n\}$ which has to be served by an (open)
distribution center. The distance between the $i^\text{th}$ client and the $j^\text{th}$ distribution center
is $d_{ij}$ ($d_{ij}\ge 0$). The maximum number of distribution centers which can be open is $q$.



## Question 1

Write a MILP model to minimize the total distance travelled by the clients. Identify the
decisions that must be taken and the corresponding decision variables that have to be
used. Identify and comment the objective function of the problem and the constraints.


**CORRECTION :**


Let $y_i = \left\{ \begin{array}{ll} 1 & \text{if distribution center number $j$ is opened}\\ 0 & \text{otherwise}\end{array} \right .~~$ for all $i \in \{1,\dots, m\}$.

Let $x_{ij} = \left\{ \begin{array}{ll} 1 & \text{if client number $j$ is served by distribution center number $i$}\\ 0 & \text{otherwise}\end{array} \right .~~$ for all $i \in \{1,\dots, m\}$ and $j \in \{1,\dots, n\}$.



The problem may be formulated as follows.
$$
\begin{align}
&\min \sum_{i = 1}^m \sum_{j = 1}^n d_{ij}x_{ij}\\
&\sum_{i=1}^n y_{i}  \le q\\
&\sum_{i=1}^n x_{ij}  = 1 & \quad & \forall j=1,\dots,n\\
& x_{ij} \le y_i & & \forall i=1,\dots,m, \forall j=1,\dots,n\\
& y_i \in \{0, 1\} & & \forall i=1,\dots,m\\
& x_{ij} \ge 0 & & \forall i=1,\dots,m, \forall j=1,\dots,n
\end{align}
$$

## Question 3

Now consider the follow case in which there are 4 potential locations ($m = 4$) and 4
clients ($n = 4$), a maximum number of 2 distribution centers can be open ($q = 2$). The
distances between the clients and the distribution centers are:

 &nbsp;    | Client 1 | Client 2 | Client 3 | Client 4
---|---|----|---|---|
Distribution center 1 | 0 | 10 | 30 | 40
Distribution center 2 | 10 | 0 | 5 | 25
Distribution center 3 | 30 | 5 | 0 | 50
Distribution center 4 | 40 | 25 | 50 | 0



Write the MILP and implement it for this specific instance.


In [ ]:
q = 2
distances = [
    [ 0, 10, 30, 40],
    [10,  0,  5, 25],
    [30,  5,  0, 50],
    [40, 25, 50,  0]
]

In [ ]:
%pip install mip

In [ ]:
from mip import *

**CORRECTION :**


MILP for the specific instance

$$
\begin{align}
& \min 10 x_{12} + 30 x_{13} + 40 x_{14} + 10 x_{21} + 5 x_{23} + 25 x_{24} + 30 x_{31} + 5 x_{32} + 50 x_{33} + 40 x_{41} + 25 x_{42} + 50 x_{43}\\
& y_1 + y_2 + y_3 + y_4 \le 2\\
&x_{11} + x_{12} + x_{13} + x_{14} = 1\\
&x_{21} + x_{22} + x_{23} + x_{24} = 1\\
&x_{31} + x_{32} + x_{33} + x_{34} = 1\\
&x_{41} + x_{42} + x_{43} + x_{44} = 1\\
&x_{11} \le y_1, x_{12} \le y_1, x_{13} \le y_1, x_{14} \le y_1,\\
&x_{21} \le y_1, x_{22} \le y_1, x_{23} \le y_1, x_{24} \le y_2,\\
&x_{31} \le y_1, x_{32} \le y_1, x_{33} \le y_1, x_{34} \le y_3,\\
&x_{41} \le y_1, x_{42} \le y_1, x_{43} \le y_1, x_{44} \le y_4,\\
&x \geq \mathbf{0}\\
&y_1,\dots,y_4 \in \{0,,1\}
\end{align}
$$

In [ ]:
##################
#   Correction   #
##################


def distribution_centers(q, distances):
    m = Model()
    I = range(len(distances))
    J = range(len(distances[0]))
    x = [
            [m.add_var() for j in J] for i in I
    ]
    y = [ m.add_var(var_type = BINARY) for i in I]
    m.objective = minimize(xsum(distances[i][j] * x[i][j] for i in I for j in J))

    # Open q distribution centers
    m += xsum(y[i] for i in I) <= q

    # for each client
    for j in J:
        m += xsum(x[i][j] for i in I) == 1

    # for each distribution center and each client
    for i in I:
        for j in J:
            m += x[i][j] <= y[i]
            
    m.verbose = 0 # to remove the output of the solution process
    status = m.optimize()
    print(f"Total distance: {round(m.objective_value)}")
    for i in I:
        if round(y[i].x) == 1:
            clients = [j+1 for j in J if round(x[i][j].x) == 1]
            print(f"Distribution center {i+1} is opened. Clients {clients} are served from it")

distribution_centers(q, distances)      